In [ ]:
import argparse
import os
import random
import sys
from pathlib import Path
from random import shuffle

import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage
import yaml
from cellpose import models
from matplotlib import pyplot as plt
from phenoscapes.cli import load_default_config, process_sample
from phenoscapes.feature_extraction import extract_features
from phenoscapes.sc import convert_to_h5ad, plot_summary
from phenoscapes.segmentation import generate_label_overlay, run_cellpose
from phenoscapes.utils import get_metadata, scale_image
from skimage import io
from skimage.color import label2rgb
from skimage.exposure import rescale_intensity
from skimage.measure import regionprops
from tqdm import tqdm

In [ ]:
os.chdir("/cluster/project/treutlein/DATA/imaging/4i_Data/Brain_ECM_4i_2_v2/")
dir_output = os.getcwd()

regex_exp = r"Stitched_C(?P<cycle_id>\d{1,2})_R(?P<well_id>\d{2,3})_ch(?P<channel_id>\d{2})\.tif$"
# load default config file
config = load_default_config()
# add regex to config
config["regex"] = regex_exp
# add path to stain metadata to config
config["stain_metadata"] = str(Path(dir_output, "stain_metadata.csv"))
# set nuclei channel
config["channel_nuclei"] = 0
# set staining channels
config["channel_stains"] = [1, 2, 3]
# set stain nuclei
config["stain_nuclei"] = "DAPI"
# Set ref cycle
config["reference_cycle"] = 0
# Set alignemnt ref cycles
config["alignment"]["reference_cycles"] = None
# Set parameter maps
config["alignment"]["param_maps"] = {
    "rigid": "/cluster/project/treutlein/USERS/gutgi/github/4i_framework/example_data/param_maps/translation.txt",
    "affine": "/cluster/project/treutlein/USERS/gutgi/github/4i_framework/example_data/param_maps/affine.txt",
}
# Cycle DAPI
config["simple_masking"]["sigma"] = 50
config["simple_masking"]["n_binary_operations"] = 50

config["refined_masking"]["cycle_dapi"] = 0
config["refined_masking"]["sigma"] = 50
config["refined_masking"]["n_binary_operations"] = 30
config["refined_masking"]["use_only_dapi"] = True
config["cropping_denoising"]["n_processes"] = 3

config["analysis_pipeline"]["slice_step"] = 1000
config["cropping_denoising"]["mask_zeros_smo"] = True
config["analysis_pipeline"]["stat"] = "median"
config["speckle_removal"]["percentile"] = 3
config["feature_extraction"]["expand_labels"] = None
# config['bg_subtraction']['cycle_dapi'] = 0
config["cellpose"]["segment_channel"] = 2
config["cellpose"]["segment_cycle"] = 1

# Only run masking
config["stitching"]["run"] = True
config["simple_masking"]["run"] = True
config["alignment"]["run"] = True
config["refined_masking"]["run"] = True
config["alignment_check"]["run"] = True
config["cropping_denoising"]["run"] = True
config["speckle_removal"]["run"] = True
config["bg_subtraction"]["run"] = True
config["cellpose"]["run"] = True
config["feature_extraction"]["run"] = True
config["analysis_pipeline"]["run"] = True
config["analysis_pipeline"]["run"] = True
config["feature_morphology_extraction"]["run"] = True
default_config = config

df_stains = pd.read_csv("stain_metadata.csv")

In [ ]:
dir_bg_subtracted = Path(dir_output, "bg_subtracted")

In [ ]:
samples = [  # Day 7
    "R066_0",
    "R062_0",
    "R073_0",
    "R072_0",
    # Day 15
    #'R021_0',
    "R026_0",
    "R022_0",
    "R024_0",
    "R049_0",
    "R055_0",
    "R044_0",
    "R057_0",
    # Day 21
    "R076_1",
    "R087_0",
    "R082_0",
    "R108_1",
    "R112_1",
    "R113_1",
    "R115_0",
]

In [ ]:
dir_images = dir_bg_subtracted
for sample in samples:
    # Create and shuffle color map and use to color compartements
    random.seed(10)
    dir_segmented_cytoplasma = Path(dir_output, "segmented_cytoplasma")
    dir_segmented_ec_niche = Path(dir_output, "segmented_ecm_niche")
    dir_segmented_nuclei = Path(dir_output, "segmented_cell_nuclei")
    dirs_segmented = [
        dir_segmented_cytoplasma,
        dir_segmented_ec_niche,
        dir_segmented_nuclei,
    ]
    img_label = io.imread(Path(dir_segmented_cytoplasma, sample + ".tif"))
    out_image = np.zeros(img_label.shape + (3,))
    mycmap = mcolors.LinearSegmentedColormap.from_list(
        "custom", ["white", "#525252"], N=20, gamma=1
    )
    greys = [
        matplotlib.colors.to_rgb(mcolors.rgb2hex(mycmap(i))) for i in range(mycmap.N)
    ]
    shuffle(greys)

    for dir_segmented, i in zip(dirs_segmented, range(len(dirs_segmented))):
        img_label = io.imread(Path(dir_segmented, sample + ".tif"))

        img_overlay = label2rgb(img_label, bg_label=0, colors=greys)
        out_image[:, :, i] = np.mean(img_overlay, axis=-1)
    out_image = (out_image * 255).astype("uint8")
    io.imsave(
        Path(
            f"/cluster/project/treutlein/USERS/gutgi/gitlab/morphodynamics-of-human-brain-organoid-patterning/4i_analysis/Brain_ECM_2/compartement_analysis/4i_updated_08_2024/compartment_overlays/{sample}.png"
        ),
        out_image,
        check_contrast=False,
    )